In [11]:
!pip install opencv-python

In [1]:
import cv2

In [2]:
import numpy as np

In [3]:
from keras.utils import np_utils

In [4]:
import os

In [5]:
data_path='C:/Users/KIIT/Desktop/dataset'

In [6]:
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]
label_dict=dict(zip(categories,labels))
print(label_dict)
print(categories)
print(labels)

{'without_mask': 0, 'with_mask': 1}
['without_mask', 'with_mask']
[0, 1]


In [7]:
img_size=100
data=[]
target=[]

In [8]:
for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
    
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)
        gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        resized=cv2.resize(gray,(img_size,img_size))
        data.append(resized)
        target.append(label_dict[category])

In [9]:
data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
target=np.array(target)
new_target=np_utils.to_categorical(target)

In [10]:
np.save('data',data)
np.save('target',new_target)

In [11]:
data=np.load('data.npy')
target=np.load('target.npy')

In [12]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

In [13]:
model=Sequential()

In [14]:
model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

In [15]:
model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

In [16]:
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(50,activation='relu'))
model.add(Dense(2,activation='softmax'))

In [17]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [29]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [18]:
from sklearn.model_selection import train_test_split
train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.2)

In [19]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=20,callbacks=[checkpoint],validation_split=0.2)

Epoch 1/20
28/28 [==============================] - 46s 2s/step - loss: 0.7498 - accuracy: 0.5460 - val_loss: 0.6611 - val_accuracy: 0.5273
INFO:tensorflow:Assets written to: model-001.model\assets
Epoch 2/20
28/28 [==============================] - 62s 2s/step - loss: 0.5793 - accuracy: 0.6982 - val_loss: 0.4539 - val_accuracy: 0.7909
INFO:tensorflow:Assets written to: model-002.model\assets
Epoch 3/20
28/28 [==============================] - 51s 2s/step - loss: 0.4048 - accuracy: 0.8231 - val_loss: 0.2636 - val_accuracy: 0.9182
INFO:tensorflow:Assets written to: model-003.model\assets
Epoch 4/20
28/28 [==============================] - 46s 2s/step - loss: 0.2804 - accuracy: 0.8823 - val_loss: 0.3006 - val_accuracy: 0.8682
Epoch 5/20
28/28 [==============================] - 61s 2s/step - loss: 0.1688 - accuracy: 0.9399 - val_loss: 0.2481 - val_accuracy: 0.9227
INFO:tensorflow:Assets written to: model-005.model\assets
Epoch 6/20
28/28 [==============================] - 52s 2s/step - lo

In [32]:
print(model.evaluate(test_data,test_target))

9/9 [==============================] - 4s 413ms/step - loss: 0.2036 - accuracy: 0.9420
[0.20355169475078583, 0.9420289993286133]


In [33]:
from keras.models import load_model


In [43]:
model = load_model('model-017.model')

face_clsfr=cv2.CascadeClassifier('C:/Users/KIIT/Desktop/haarcascade_frontalface_default.xml')

source=cv2.VideoCapture(0)

labels_dict={0:'NO MASK',1:'MASK'}
color_dict={0:(0,0,255),1:(0,255,0)}

In [44]:
while(True):

    ret,img=source.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.3,5)  

    for (x,y,w,h) in faces:
    
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,1))
        result=model.predict(reshaped)

        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()